In [3]:
import urllib.request
import zipfile
import progressbar

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
# Downloads the zip of the training data and unzips it into a direcoty at horse-or-human/training
url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"


# Progress bar
pbar = None

def show_progress(block_num, block_size, total_size):
    global pbar
    if pbar is None:
        pbar = progressbar.ProgressBar(maxval=total_size)
        pbar.start()

    downloaded = block_num * block_size
    if downloaded < total_size:
        pbar.update(downloaded)
    else:
        pbar.finish()
        pbar = None


file_name = "horse-or-human.zip"
training_dir = 'horse-or-human/training/'

urllib.request.urlretrieve(url, file_name, show_progress)

zip_ref = zipfile.ZipFile(file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

100% |########################################################################|


In [ ]:
# Adding Validation to the Horses or Humans Dataset

validation_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"

validation_file_name = "validation-horse-or-human.zip"
validation_dir = "horse-or-human/validation/"
urlretrieve(validation_url, validation_file_name, show_progress) # show_progress is for progression bar

zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

In [16]:
# Generating data

train_datagen = ImageDataGenerator(rescale = 1/255)

train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size = (300, 300),
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.


In [ ]:
# Generating Validation data
validation_datagen = ImageDataGenerator(rescale = 1/255)

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size = (300, 300),
    class_mode = 'binary'
)

Validation
- To add validation, you'll need a validation dataset that's separated from the training dataset.
- In some case you'll get a master dataset that you have to split yourself

Difference between Training Data and Validation data and Testing Data
- Training data is the data that is used to teach the network how the data and labels fit together.
- Validation data is used to see how the network is doing with previously unseen data while you are training
- Test data is used after training to see how the network does with data it has never priviously seen

In [ ]:
# CNN Architecture for Horses and Humans

model = Sequential([
                    Conv2D(16, (3, 3), activation = 'relu', input_shape = (300, 300, 3)),
                    MaxPooling2D(2, 2),
                    Conv2D(32, (3, 3), activation = 'relu'),
                    MaxPooling2D(2, 2),
                    Conv2D(64, (3, 3), activation = 'relu'),
                    MaxPooling2D(2, 2),
                    Conv2D(64, (3, 3), activation = 'relu'),
                    MaxPooling2D(2, 2),
                    Conv2D(64, (3, 3), activation = 'relu'),
                    MaxPooling2D(2, 2),
                    Flatten(),
                    Dense(512, activation = 'relu'),
                    Dense(1, activation = 'sigmoid')
])

# model.summary()

# Compiling model
model.compile(loss = 'binary_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Older generations uses model.fit_generator while the later can use either fit or that
model.fit(train_generator, epochs = 15, validation_data = validation_generator)

model.evaluate(train_generator)